# Death Row
## Scraping death row inmate data and last statements

The primary goal of this notebook is to use BeautifulSoup4 to web scrap a data set from the Texas DoJ on death row inmates.

In [2]:
# Import libraries
import requests
import numpy as np
import pandas as pd
import matplotlib as mpl
import seaborn as sns
from bs4 import BeautifulSoup

### Get meta data and URLs from parent page

In [4]:
DEATHROW_PARENT_URL = 'https://www.tdcj.texas.gov/death_row/dr_executed_offenders.html'
parent_page = requests.get(DEATHROW_PARENT_URL)
if parent_page.status_code == 200:
    print("Parent page query successful.")
else:
    print("Parent page query failed.")

Parent page query successful.


### Define function to parse the parent table

In [136]:
def date2datetime(date_str):
    
    s = tuple(map(lambda x: int(x), date_str.split('/')))
    return '%d-%02.0f-%02.0f' % (s[2],s[0],s[1])

In [61]:
def parse_inmate_rows(rows):
    """
    Parse html from death row inmate table line by line and output values to dict array
    Input: rows List[Str]
    Output: inmate_table List[Dict]
    """
    
    # get column headers from the first row
    headers = rows[0].select('th')
    headers = list(map(lambda x: x.text.strip(), headers))
    headers[1] = 'info_url'
    headers[2] = 'statement_url'
    rows.pop(0)
    inmate_table = []
    
    # iterate over table rows
    for row in rows:
        
        # intialize entry for new inmate data
        inmate_dict = dict(zip(headers,['']*len(headers)))
        
        # parse data from each row
        for j,item in enumerate(row.select('td')):
            if j in [0,5,6]:
                inmate_dict[headers[j]] = int(item.text.strip())
            elif j in [1,2]:
                inmate_dict[headers[j]] = item.a['href']
            elif j == 7:
                # convert date to YYYY-MM-DD format
                inmate_dict[headers[j]] = date2datetime(item.text)
            else:
                inmate_dict[headers[j]] = item.text.strip()
        
        # add entry to inmate table
        inmate_table.append(inmate_dict)
        
    return inmate_table
        

In [67]:
# parse parent page html and format as dataframe
par_soup = BeautifulSoup(parent_page.text, 'html.parser')
inmate_rows = par_soup.select('tr')
inmate_df = pd.DataFrame(parse_inmate_rows(inmate_rows))
inmate_df.set_index('Execution',drop=True)

,info_url,statement_url,Last Name,First Name,TDCJNumber,Age,Date,Race,County
Execution,,,,,,,,,
570,dr_info/wardlowbilly.html,dr_info/wardlowbillylast.html,Wardlow,Billy,999137,45,2020-07-08,White,Titus
569,dr_info/ochoaabel.html,dr_info/ochoaabellast.html,Ochoa,Abel,999450,47,2020-02-06,Hispanic,Dallas
568,dr_info/gardnerjohn.html,dr_info/gardnerjohnlast.html,Gardner,John,999516,64,2020-01-15,White,Collin
567,dr_info/runnelstravis.html,dr_info/runnelstravislast.html,Runnels,Travis,999505,46,2019-12-11,Black,Potter
566,dr_info/halljusten.html,dr_info/halljustenlast.html,Hall,Justen,999497,38,2019-11-06,White,El Paso
...,...,...,...,...,...,...,...,...,...
5,dr_info/skillerndoyle.jpg,dr_info/skillerndoylelast.html,Skillern,Doyle,518,49,1985-01-16,White,Lubbock
4,dr_info/barefootthomas.jpg,dr_info/barefootthomaslast.html,Barefoot,Thomas,621,39,1984-10-30,White,Bell
3,dr_info/obryanronald.jpg,dr_info/obryanronaldlast.html,O'Bryan,Ronald,529,39,1984-03-31,White,Harris


### Use URLs scraped from the table to scrape additional data

In particular we are interested in two additional pages on each inmate:
1. **Inmate Information Page** - containing demographic information such as age, race, DOB, education etc.
2. **Last Statement Page** - containing text (if any) from the inmate's last statement before execution.

**Scrape Info Pages**

First we'll define a function to request html from the inmate information page and parse it into some features of interest. Although the much data on each page is stored in a table, the individual elements do not have unique CSS indentifiers that will allow us to easily grab each element. So even though it's a bit inflexible, we'll just grab all table rows parse the data by row number.

There are a few additional problems we'll have to handle as well:
- Numeric data such as age, years of eduction etc is not in a standard format
- Some records are stored as images rather than html; so those will be skipped
- Not all of the data is stored inside the table but is loosely formated into paragraghs at the bottom of the page

In [449]:
import re

# Define function to split victim race and gender since they are stored in a single field on the page.
def parse_victim_race_gender(s):
    # use regular expression to pull out any instances of race
    race = re.findall('(white|black|hispanic|other)',s.lower())
    if race:
        race = ''.join([item + ', ' for item in race])[0:-2]
    else:
        race = None
        
    # use regular expression to pull out any instances of race    
    gender = re.findall('(male|female)',s.lower())
    if gender:
        gender = ''.join([item + ', ' for item in gender])[0:-2]
    else:
        gender = None
    
    return race, gender
    
# Define function to iterate over each page URL and parse the info data
def scrape_info_pages(urls, parent_url):
    """
    Input: 
        urls List[Str]
        parent_url Str
    Output: 
        info_table List[Dict]
    """
    
    # initialize placeholder for inmate info
    info_table = []
    
    # define rows of interest in the info table
    rows_of_interest = [2,4,5,7,9,10,12,13]
    
    for j, url in enumerate(urls):
        
        # define new entry
        info = {
            'DOB': None,
            'Age': None,
            'Years_Education': None,
            'Age_Offense': None,
            'Race': None,
            'Gender': None,
            'Height': None,
            'Weight': None,
            'Prior_Occupation': None,
            'Prior_Record': None,
            'Incident_Summary': None,
            'Victim_Race': None,
            'Victim_Gender': None,
        }
        info_keys = list(info.keys())
        
        # check if URL is for html or jpg
        print('Requesting page %d of %d...' % (len(urls)-j,len(urls)), end='')
        if url.split('.')[1] == 'html':
            page = requests.get(parent_url+'/'+url)
            if page.status_code == 200:
                print('success.\n')
            else:
                info_table.append(info)
                print('failed.\n')
                continue
                
                
            # parse html
            soup = BeautifulSoup(page.text, 'html.parser')
            rows = soup.find_all('td',class_='table_deathrow_align_left')
            if not rows:
                info_table.append(info)
                continue
            
            # assign each row manually since our table items don't have CSS identifiers
            for i, row_idx in enumerate(rows_of_interest):
                row_str = rows[row_idx].text
                if row_idx == 2:
                    row_str = date2datetime(row_str)
                elif row_idx in [4,5,7,12,13]:
                    row_str = ''.join([c for c in row_str if c.isdigit()])
                    if row_str.isnumeric():
                        if row_idx == 12:
                            row_str = int(row_str[0])*12 + int(row_str[:1])
                        else:
                            row_str = int(row_str)
                    else:
                        row_str = np.nan
                info[info_keys[i]] = row_str

            all_para = soup.find_all('p')

            for p in all_para:
                span = p.select('span')
                p_text = re.findall('(?<=\n).*(?=\n)*',p.text)
                if not span or not p_text:
                    continue
                    
                span_title = span[0].text
                p_text = p_text[0].strip()
                if span_title == 'Prior Occupation':
                    info['Prior_Occupation'] = p_text
                elif span_title == 'Prior Prison Record':
                    info['Prior_Record'] = p_text
                elif span_title == 'Summary of Incident':
                    info['Incident_Summary'] = p_text
                elif span_title == 'Race and Gender of Victim':
                    info['Victim_Race'], info['Victim_Gender'] = parse_victim_race_gender(p_text)
        else:
            print('Skipping page %d. Not HTML format.\n')
         
        # append new item to output table
        info_table.append(info)
        
    return info_table
                    

In [450]:
# query each inmate info URL and parse into values
import os
path, ext = os.path.splitext(DEATHROW_PARENT_URL)
path = ''.join([s + '/' for s in path.split('/')[0:-1]])
urls = inmate_df['info_url'].tolist()
info_page_dict = scrape_info_pages(urls, path)

Requesting page 570 of 570...success.

Requesting page 569 of 570...success.

Requesting page 568 of 570...success.

Requesting page 567 of 570...success.

Requesting page 566 of 570...success.

Requesting page 565 of 570...success.

Requesting page 564 of 570...success.

Requesting page 563 of 570...success.

Requesting page 562 of 570...success.

Requesting page 561 of 570...success.

Requesting page 560 of 570...Skipping page %d. Not HTML format.

Requesting page 559 of 570...Skipping page %d. Not HTML format.

Requesting page 558 of 570...success.

Requesting page 557 of 570...success.

Requesting page 556 of 570...Skipping page %d. Not HTML format.

Requesting page 555 of 570...success.

Requesting page 554 of 570...success.

Requesting page 553 of 570...success.

Requesting page 552 of 570...success.

Requesting page 551 of 570...success.

Requesting page 550 of 570...success.

Requesting page 549 of 570...success.

Requesting page 548 of 570...success.

Requesting page 547 of 57

Requesting page 389 of 570...success.

Requesting page 388 of 570...success.

Requesting page 387 of 570...Skipping page %d. Not HTML format.

Requesting page 386 of 570...success.

Requesting page 385 of 570...Skipping page %d. Not HTML format.

Requesting page 384 of 570...success.

Requesting page 383 of 570...success.

Requesting page 382 of 570...success.

Requesting page 381 of 570...Skipping page %d. Not HTML format.

Requesting page 380 of 570...success.

Requesting page 379 of 570...Skipping page %d. Not HTML format.

Requesting page 378 of 570...Skipping page %d. Not HTML format.

Requesting page 377 of 570...Skipping page %d. Not HTML format.

Requesting page 376 of 570...Skipping page %d. Not HTML format.

Requesting page 375 of 570...success.

Requesting page 374 of 570...success.

Requesting page 373 of 570...success.

Requesting page 372 of 570...success.

Requesting page 371 of 570...Skipping page %d. Not HTML format.

Requesting page 370 of 570...Skipping page %d. Not 

Requesting page 234 of 570...success.

Requesting page 233 of 570...Skipping page %d. Not HTML format.

Requesting page 232 of 570...Skipping page %d. Not HTML format.

Requesting page 231 of 570...Skipping page %d. Not HTML format.

Requesting page 230 of 570...Skipping page %d. Not HTML format.

Requesting page 229 of 570...Skipping page %d. Not HTML format.

Requesting page 228 of 570...Skipping page %d. Not HTML format.

Requesting page 227 of 570...Skipping page %d. Not HTML format.

Requesting page 226 of 570...Skipping page %d. Not HTML format.

Requesting page 225 of 570...Skipping page %d. Not HTML format.

Requesting page 224 of 570...Skipping page %d. Not HTML format.

Requesting page 223 of 570...Skipping page %d. Not HTML format.

Requesting page 222 of 570...Skipping page %d. Not HTML format.

Requesting page 221 of 570...Skipping page %d. Not HTML format.

Requesting page 220 of 570...Skipping page %d. Not HTML format.

Requesting page 219 of 570...Skipping page %d. Not 

Requesting page 66 of 570...success.

Requesting page 65 of 570...Skipping page %d. Not HTML format.

Requesting page 64 of 570...Skipping page %d. Not HTML format.

Requesting page 63 of 570...Skipping page %d. Not HTML format.

Requesting page 62 of 570...Skipping page %d. Not HTML format.

Requesting page 61 of 570...Skipping page %d. Not HTML format.

Requesting page 60 of 570...Skipping page %d. Not HTML format.

Requesting page 59 of 570...Skipping page %d. Not HTML format.

Requesting page 58 of 570...Skipping page %d. Not HTML format.

Requesting page 57 of 570...Skipping page %d. Not HTML format.

Requesting page 56 of 570...Skipping page %d. Not HTML format.

Requesting page 55 of 570...Skipping page %d. Not HTML format.

Requesting page 54 of 570...success.

Requesting page 53 of 570...success.

Requesting page 52 of 570...Skipping page %d. Not HTML format.

Requesting page 51 of 570...success.

Requesting page 50 of 570...success.

Requesting page 49 of 570...Skipping page 

In [455]:
# convert dict to dataframe and add to inmate_df
info_page_df = pd.DataFrame(info_page_dict)
inmate_df = pd.concat([inmate_df, info_page_df], axis=1)
inmate_df.head(5)

,Execution,info_url,statement_url,Last Name,First Name,TDCJNumber,Age,Date,Race,County,...,Age_Offense,Race,Gender,Height,Weight,Prior_Occupation,Prior_Record,Incident_Summary,Victim_Race,Victim_Gender
0,570,dr_info/wardlowbilly.html,dr_info/wardlowbillylast.html,Wardlow,Billy,999137,45,2020-07-08,White,Titus,...,18.0,White,Male,78.0,204.0,Laborer,None,"On June 14, 1993, Wardlow and the co-defendant...",white,male
1,569,dr_info/ochoaabel.html,dr_info/ochoaabellast.html,Ochoa,Abel,999450,47,2020-02-06,Hispanic,Dallas,...,29.0,Hispanic,Male,65.0,182.0,Heavy Equipment Operator/Laborer,None,"On August 4, 2002, in Dallas, Texas, Ochoa fa...","hispanic, hispanic","female, male"
2,568,dr_info/gardnerjohn.html,dr_info/gardnerjohnlast.html,Gardner,John,999516,64,2020-01-15,White,Collin,...,49.0,White,Male,78.0,190.0,Painter,MS Dept of Corrections #55484 on an eight yea...,"On January 23, 2005 in Collin County, Gardner...",white,female
3,567,dr_info/runnelstravis.html,dr_info/runnelstravislast.html,Runnels,Travis,999505,46,2019-12-11,Black,Potter,...,30.0,Black,Male,78.0,208.0,Laborer,TDCJ# 081244 on a five-year sentence for burg...,"On January 29, 2003, in Potter County, whil...",white,male
4,566,dr_info/halljusten.html,dr_info/halljustenlast.html,Hall,Justen,999497,38,2019-11-06,White,El Paso,...,21.0,White,Male,78.0,197.0,Laborer,TDCJ# 914053 on a 2 year sentence from El ...,"On October 28, 2002, in El Paso County, Tex...",None,female


In [475]:
import unicodedata

# Define function to scrape inmate last statements from each Last Statement page
def scrape_last_statements(urls, parent_url):
    """
    Input: 
        urls List[Str]
        parent_url Str
    Output: 
        info_table List[Str]
    """
    
    # initialize placeholder for inmate info
    statements = []
    
    for j, url in enumerate(urls):
        
        statement = ''
        
        # check if URL is for html or jpg
        print('Requesting page %d of %d...' % (len(urls)-j,len(urls)), end='')
        if url.split('.')[1] == 'html':
            page = requests.get(parent_url+'/'+url)
            if page.status_code == 200:
                print('success.\n')
            else:
                statements.append(statement)
                print('failed.\n')
                continue
         
        # parse html and iterate over all <p> tags to find Last statement
        soup = BeautifulSoup(page.text,'html.parser')
        all_para = soup.select('p')
        num_para = len(all_para)
        for i, p in enumerate(all_para):
            if 'last statement' in p.text.lower():
                if i+1 < num_para:
                    # remove unicode formatting characters
                    statement = unicodedata.normalize('NFKC',all_para[i+1].text).strip()
                break
        
        # add to output list
        statements.append(statement)
        
    return statements
    

In [476]:
# query each last statement URL and parse into values
urls = inmate_df['statement_url'].tolist()
statements = scrape_last_statements(urls, path)
inmate_df['last_statement'] = statements

Requesting page 570 of 570...success.

Requesting page 569 of 570...success.

Requesting page 568 of 570...success.

Requesting page 567 of 570...success.

Requesting page 566 of 570...success.

Requesting page 565 of 570...success.

Requesting page 564 of 570...success.

Requesting page 563 of 570...success.

Requesting page 562 of 570...success.

Requesting page 561 of 570...success.

Requesting page 560 of 570...success.

Requesting page 559 of 570...success.

Requesting page 558 of 570...success.

Requesting page 557 of 570...success.

Requesting page 556 of 570...success.

Requesting page 555 of 570...success.

Requesting page 554 of 570...success.

Requesting page 553 of 570...success.

Requesting page 552 of 570...success.

Requesting page 551 of 570...success.

Requesting page 550 of 570...success.

Requesting page 549 of 570...success.

Requesting page 548 of 570...success.

Requesting page 547 of 570...success.

Requesting page 546 of 570...success.

Requesting page 545 of 57

Requesting page 150 of 570...success.

Requesting page 149 of 570...success.

Requesting page 148 of 570...success.

Requesting page 147 of 570...success.

Requesting page 146 of 570...success.

Requesting page 145 of 570...success.

Requesting page 144 of 570...success.

Requesting page 143 of 570...success.

Requesting page 142 of 570...success.

Requesting page 141 of 570...success.

Requesting page 140 of 570...success.

Requesting page 139 of 570...success.

Requesting page 138 of 570...success.

Requesting page 137 of 570...success.

Requesting page 136 of 570...success.

Requesting page 135 of 570...success.

Requesting page 134 of 570...success.

Requesting page 133 of 570...success.

Requesting page 132 of 570...success.

Requesting page 131 of 570...success.

Requesting page 130 of 570...success.

Requesting page 129 of 570...success.

Requesting page 128 of 570...success.

Requesting page 127 of 570...success.

Requesting page 126 of 570...success.

Requesting page 125 of 57

### Export data to CSV

Exporting the data to file so we can use it for EDA and NLP classification of last statements in future notebooks.

In [483]:
inmate_df.to_csv('inmate_table.csv')